# Statistical Model Construction
Optimizing BepiPred performance based on errors in predictions

In [5]:
import json
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import scipy.stats
import numpy as np
import utils

In [6]:
# Load in data
with open('../bepipred3.json') as fi:
    data = json.load(fi)

In [7]:
pdbs = dict()

for chain in data:
    pdbid, chainid, partition = chain['desc'].split(' ')

    if pdbid not in pdbs:
        pdbs[pdbid] = {'PDB': list(),'predictions': list(),'targets': list(),'aminoacid':list(), 'partition': list() }


    preds = chain['preds']
    pdbs[pdbid]['predictions'].extend(preds)
    pdbs[pdbid]['targets'].extend(chain['epitope'])
    pdbs[pdbid]['aminoacid'].extend(list(chain['seq'])) 
    pdbs[pdbid]['partition'].extend([ int(partition) for x in range(len(chain['preds'])) ])
    pdbs[pdbid]['PDB'].extend([ pdbid for x in range(len(chain['preds'])) ])


In [8]:
pdbs_dataframe = pd.DataFrame.from_dict({'PDB':list(), 'predictions': list(),'targets': list(),'aminoacid':list(), 'partition': list() })
for pdb in sorted(pdbs):
    dataframe = pd.DataFrame.from_dict(pdbs[pdb])
    pdbs_dataframe = pdbs_dataframe.append(dataframe)

epitopes = pdbs_dataframe.groupby('PDB').targets.sum()
pred_sum = pdbs_dataframe.groupby('PDB').predictions.sum()
scipy.stats.pearsonr(epitopes, pred_sum)

ValueError: arrays must all be same length

# Prepare input for stan model

Input features:
- Prediction score
- Amino Acid Frequency within protein
- 1/protein length, for each protein
- Protein class label
- Amino Acid class label

### Add Amino acid Index

In [10]:
df_tester = pdbs_dataframe.copy(deep=True)
#Add Amino Acid group Index
aminos = 'ARNDCQEGHILKMFPSTWYV'
AA2ind = dict(zip(list(aminos),range(0,len(aminos))))
df_tester['Ind'] = [AA2ind[AA] for AA in df_tester['seq'].values]

KeyError: 'seq'

In [ ]:
g = df_tester.groupby('desc')[['epitope','preds']].sum()
g.values
scipy.stats.pearsonr(g.values[:,0],g.values[:,1])
sns.regplot(x='epitope',y='preds',data=g)

### Get Prior amino acid frequency for epitopes

In [11]:
def getAminoCount(df,aaCol = 'seq', aminos='ARNDCQEGHILKMFPSTWYV'):
    seq = ''.join(df[aaCol].values)
    return sorted([(aa,seq.count(aa)) for aa in list(aminos)])

def getAminoFreq(df,aaCol = 'seq', aminos='ARNDCQEGHILKMFPSTWYV'):
    seq = ''.join(df[aaCol].values)
    seqLen = float(len(seq))
    return dict([(aa,seq.count(aa)/seqLen) for aa in list(aminos)])

df_tester_epi = df_tester[df_tester['epitope']==0]
priorEpiAAfreq = getAminoFreq(df_tester_epi)
priorEpiAAfreq

KeyError: 'epitope'

In [ ]:
def getAminoCount(df,aaCol = 'seq', aminos='ARNDCQEGHILKMFPSTWYV'):
    seq = ''.join(df[aaCol].values)
    return sorted([(aa,seq.count(aa)) for aa in list(aminos)])

def getCountMat(dfIn):
    aminoCountList = []
    for desc, df in dfIn.groupby('desc'):
        if df.epitope.sum() < 30: #Hack way to select only proteins with one epitope
            aminoCounts = getAminoCount(df)
            AAs, counts = zip(*aminoCounts)
            aminoCountList.append(counts)
    return np.array(aminoCountList)

df_tester_nonEpi = df_tester[df_tester['epitope']==0]
aminoCountMat = getCountMat(df_tester_nonEpi)

In [12]:
###Hierarchical Logistic regression model for a protein

code ="""
data {
  int<lower=1> D;//Number of input variables
  int<lower=0> N;//Number of samples
  int<lower=0> L;//Number of levels(amino acids)
  int<lower=0,upper=1> y[N];//Target: epitope
  int<lower=0,upper=L> ll[N];//Level indicator
  row_vector<lower=0,upper=1>[D] x[N];//Feature: bepipred prediction
  //Test data
  //int<lower=0> N_test;//Number of samples
  //int<lower=1,upper=L> ll_test[N_test];//Level indicator
  //row_vector[D] x_test[N_test];//Feature: bepipred prediction
}
parameters {
  real mu[D];
  real<lower=0> sigma[D];
  vector[D] beta[L];
  real mu0prior;
  real<lower=0> s0prior;
}
model {
  for (d in 1:D) {
    mu[d] ~ normal(mu0prior, s0prior);
    for (l in 1:L)
      beta[l,d] ~ normal(mu[d], sigma[d]);
  }
  for (n in 1:N)
    y[n] ~ bernoulli(inv_logit(x[n] * beta[ll[n]]));
}
//generated quantities {
//  int<lower=0,upper=1> epitope_test[N_test];
//  for (nn in 1:N_test)
//      epitope_test[nn] = bernoulli(inv_logit(x_test[nn] * beta[ll_test[nn]]));
//    }
"""

logisticProtein = pystan.StanModel(model_code=code)

NameError: name 'pystan' is not defined

In [ ]:
def makeInpArray(preds,D=2):
    l = len(preds)
    inp = np.ones((l,D))
    inp[:,D-1] = preds
    return inp

testDat = df_tester[df_tester['desc']=='4G3Y_HL C 1']
trainProt = list(set(df_tester.desc.values))[:1]#Select only the first set of proteins
trainDat = df_tester[[desc in trainProt for desc in df_tester.desc.values]]

trainArray = makeInpArray(trainDat.preds.values,D=2)
testArray = makeInpArray(testDat.preds.values)

data = dict(
    D = trainArray.shape[1],
    N = len(trainDat),
    L = len(set(trainDat.Ind.values)),
    y = [int(x) for x in trainDat.epitope.values],
    ll = trainDat.Ind.values,
    x = trainArray,
    
    #Test data
    #x_test = testArray,
    #N_test=len(testDat),
    #ll_test = testDat.Ind.values
)

fit = logisticProtein.sampling(data=data, iter=5000, chains=2)
samples = fit.extract(permuted=True)

In [ ]:
fit.plot()

In [ ]:
s,a,p=samples['beta'].shape
for i in range(p):
    for j in range(a):
        sns.distplot(samples['beta'][:,j,i],hist=False,label=list(aminos)[j])
    plt.show()

In [ ]:
from scipy.special import expit  # aka logistic

#trainDat = df_tester[df_tester['desc']=='4HJ0_CD B 1']
trainDat = df_tester[[desc in trainProt for desc in df_tester.desc.values]]
trainArray = makeInpArray(trainDat.preds.values)

trainDat

g = 20
h = 0

for i in range(0,len(aminos)):
    #Plot alpha distribution
    alphas = samples['beta'][h:h+g,i,0]
    sns.distplot(alphas,hist=False,kde=True)
    plt.show()
    #Plot beta distribution
    betas = samples['beta'][h:h+g,i,1]
    sns.distplot(betas,hist=False,kde=True)
    plt.show()
    #Show input dataframe
    aaDF = trainDat[trainDat['Ind']==i+1]
    aaDF = aaDF.sort_values('preds')
    print(aaDF)
    #Show logistic regressions
    x=np.linspace(-1,1,g)
    fs = expit(alphas[:,None] + betas[:,None]*x)
    plt.plot(x,fs.T)
    plt.show()


In [ ]:
###Basic regression model for a protein

code ="""
data {
  int<lower=0> N;//Number of samples
  int<lower=0> L;//Number of levels(amino acids)
  real<lower=0,upper=1> y[N];//Target: epitope
  int<lower=0,upper=L> ll[N];//Level indicator
  real<lower=0,upper=1> x[N];//Feature: bepipred prediction
  //Test data
  //int<lower=0> N_test;//Number of samples
  //int<lower=1,upper=L> ll_test[N_test];//Level indicator
  //row_vector[D] x_test[N_test];//Feature: bepipred prediction
}
parameters {
  //real mu[L];
  real<lower=0> sigma;
  real beta[L];
  real alpha[L];
}
model {
  for (n in 1:N)
    y[n] ~ normal(alpha[ll[n]] + beta[ll[n]]*x[n], sigma);
}
//generated quantities {
//  int<lower=0,upper=1> epitope_test[N_test];
//  for (nn in 1:N_test)
//      epitope_test[nn] = bernoulli(inv_logit(x_test[nn] * beta[ll_test[nn]]));
//    }
"""

sm = pystan.StanModel(model_code=code)

In [ ]:
def makeInpArray(preds,D=2):
    l = len(preds)
    inp = np.ones((l,D))
    inp[:,D-1] = preds
    return inp

testDat = df_tester[df_tester['desc']=='4G3Y_HL C 1']
trainProt = list(set(df_tester.desc.values))[:1]
#trainDat = df_tester[df_tester['desc']=='4HJ0_CD B 1']
trainDat = df_tester[[desc in trainProt for desc in df_tester.desc.values]]


trainArray = makeInpArray(trainDat.preds.values,D=2)
testArray = makeInpArray(testDat.preds.values)

data = dict(
    D = trainArray.shape[1],
    N = len(trainDat),
    L = len(set(trainDat.Ind.values)),
    y = [int(x) for x in trainDat.epitope.values],
    ll = trainDat.Ind.values,
    x = trainDat.preds.values,
    
    #Test data
    #x_test = testArray,
    #N_test=len(testDat),
    #ll_test = testDat.Ind.values
)

fit = sm.sampling(data=data, iter=5000, chains=2)
samples = fit.extract(permuted=True)

In [13]:
###Code to learn the trivial multinomial of amino acid frequency

code ="""
data {
    int num_rows;
    int num_cols;
    int y[num_rows, num_cols];
 }
parameters {
    simplex[num_cols] theta;
}
model {
    for (i in 1:num_rows) {
        y[i,] ~ multinomial(theta);
    } 
    // some prior on theta
}
"""

aaFreq = pystan.StanModel(model_code=code)

NameError: name 'pystan' is not defined

In [ ]:
data = dict(
    num_rows=aminoCountMat.shape[0],
    num_cols=aminoCountMat.shape[1],
    y=aminoCountMat
)

fit = sm.sampling(data=data, iter=5000, chains=2)
samples = fit.extract(permuted=True)

In [ ]:
aminos = 'ARNDCQEGHILKMFPSTWYV'
aminosSorted = ''.join(sorted(list(aminos)))

aminosSortedTup = list(zip(aminosSorted ,fit['theta'].mean(axis=0)))
AAfreqDFstan = pd.DataFrame(aminosSortedTup,columns=['AA','Freq'])
AAfreqDFstan['Method'] = 'Stan'

In [ ]:
AAfreqDFcount = pd.DataFrame(priorEpiAAfreq.items(),columns=['AA','Freq'])
AAfreqDFcount['Method'] = 'Count'
AAfreqDF = pd.concat([AAfreqDFcount,AAfreqDFstan])
sns.barplot(x='AA',y='Freq',hue='Method', data=AAfreqDF)
